In [ ]:
import numpy as np
import sys

In [ ]:
'''
--------------device parameters in units of 2*pi---------------

             qubit0----qubit1----qubit2
                    g0        g1
'''

# qubit frequencies: E_1 - E_0
omega = [5.100, 8.100, 6.200]

# anharmonicity: (E_2 - E_1)-(E_1 - E_0)
alpha = [-0.310, -0.235, -0.285]/2

# effective coupling strength
lg = [0.146, 0.164]

In [ ]:
def eff_ham_mpo(omega, alpha, lg):
    '''
    effective model Hamiltonian for architecture I -- Eq.31
    '''
    # bosonic creation operator
    bt = np.diag([1., np.sqrt(2)], k=-1)
    # bosonic creation operator
    bn = bt.transpose()
    # number operator
    n = np.kron(bt, bn)
    # charge operator
    q = bt + bn
    # identity
    I = np.eye(3)
    # null matrix
    O = np.zeros((3,3))
    
    onsite = [w*n + a*n@(a-I) for w,a in zip(omega, alpha)]
    inter = [g*q for g in lg]

    W0 = np.array([[onsite[0], inter[0], I]])
    W1 = np.array([
        [I, O, O],
        [q, O, O],
        [onsite[1], inter[q], I]
    ])
    W2 = np.array([
        [I],
        [q],
        [onsite[2]]
    ])
    
    return W1, W2, W3

In [ ]:
# initial state z=(1, 0, 0)


# maximally allowed virtual bond dimensions
D = [4,4]
psi = ptn.MPS(mpoH.qd, [Di*[0] for Di in D], fill='random')
# effectively clamp virtual bond dimension of initial state
Dinit = 8
for i in range(L):
    psi.A[i][:, Dinit:, :] = 0
    psi.A[i][:, :, Dinit:] = 0
psi.orthonormalize(mode='left')

# time step can have both real and imaginary parts;
# for real time evolution use purely imaginary dt!
dt = 0.01 - 0.05j
numsteps = 100

# run TDVP time evolution
ptn.integrate_local_singlesite(mpoH, psi, dt, numsteps, numiter_lanczos=5)
# psi now stores the (approximated) time-evolved state exp(-dt*numsteps H) psi